# 📘 Архитектуры Агентов 10: Симулятор / Ментальная Модель в Цикле (Simulator / Mental-Model-in-the-Loop)

Добро пожаловать в десятый блокнот нашей серии. Сегодня мы исследуем сложную архитектуру, разработанную для безопасности и надежного принятия решений в средах с высокими ставками: **Simulator** (Симулятор), также известную как **Mental-Model-in-the-Loop** (Ментальная Модель в Цикле).

Основная идея состоит в том, чтобы дать агенту возможность "подумать, прежде чем действовать" очень конкретным образом. Вместо того чтобы немедленно совершать действие в реальном мире, агент сначала тестирует свое предполагаемое действие во внутренней, смоделированной версии среды. Наблюдая за вероятными последствиями в этой безопасной "песочнице", он может оценить риски, уточнить свою стратегию и только затем выполнить более обдуманное действие в реальности.

Чтобы продемонстрировать это, мы создадим простого **агента по торговле акциями**. "Реальным миром" будет служить симулятор рынка, который продвигается на один шаг за раз. Перед совершением сделки наш агент будет:
1.  Предлагать общую стратегию (например, "покупать агрессивно").
2.  Прогонять эту стратегию через *форкнутую* (ответвленную) версию рыночного симулятора на несколько шагов в будущее, чтобы увидеть потенциальные результаты.
3.  Анализировать результаты симуляции для оценки рисков и вознаграждения.
4.  Принимать окончательное, уточненное решение (например, "Симуляция показывает высокую волатильность; давайте купим меньшее количество").
5.  Выполнять эту уточненную сделку на реальном рынке.

Этот паттерн имеет решающее значение для перехода агентов от информационных задач к выполнению действий в реальном мире, где ошибки могут иметь реальные последствия.

### Определение
Архитектура **Simulator** или **Mental-Model-in-the-Loop** подразумевает агента, который использует внутреннюю модель своей среды для симуляции исходов потенциальных действий перед их выполнением. Это позволяет агенту проводить анализ "что-если", предвидеть последствия и уточнять свой план для безопасности и эффективности.

### Высокоуровневый рабочий процесс

1.  **Наблюдение (Observe):** Агент наблюдает за текущим состоянием реальной среды.
2.  **Предложение действия (Propose Action):** Основываясь на своих целях и текущем состоянии, модуль планирования агента генерирует высокоуровневое предполагаемое действие или стратегию.
3.  **Симуляция (Simulate):** Агент делает "форк" (копию) текущего состояния среды в изолированную симуляцию ("песочницу"). Он применяет предложенное действие и запускает симуляцию вперед, чтобы наблюдать ряд возможных исходов.
4.  **Оценка и Уточнение (Assess & Refine):** Агент анализирует результаты симуляции. Привело ли действие к желаемому результату? Были ли непредвиденные негативные последствия? Основываясь на этой оценке, он уточняет свое первоначальное предложение в окончательное, конкретное действие.
5.  **Выполнение (Execute):** Агент выполняет окончательное, уточненное действие в *реальной* среде.
6.  **Повторение (Repeat):** Цикл начинается снова с нового состояния реальной среды.

### Когда использовать / Применения
*   **Робототехника:** Симуляция захвата или траектории движения перед перемещением физического манипулятора, чтобы избежать столкновений или повреждений.
*   **Принятие решений с высокими ставками:** В финансах — симуляция влияния сделки на портфель при различных рыночных условиях. В здравоохранении — симуляция потенциальных эффектов плана лечения.
*   **ИИ в сложных играх:** ИИ в стратегической игре, симулирующий несколько ходов вперед, чтобы выбрать оптимальный.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Безопасность и снижение рисков:** Значительно снижает вероятность вредных или дорогостоящих ошибок, предварительно проверяя действия в безопасной среде.
    *   **Улучшенная производительность:** Приводит к более надежным и взвешенным решениям, позволяя заглядывать вперед и планировать.
*   **Слабые стороны:**
    *   **Разрыв между симуляцией и реальностью:** Эффективность полностью зависит от точности (фиделити) симулятора. Если модель мира неточна, планы агента могут быть основаны на ложных предположениях.
    *   **Вычислительные затраты:** Запуск симуляций, особенно множества сценариев, вычислительно дорог и медленнее, чем прямое действие.

## Фаза 0: Основа и Настройка

Мы установим библиотеки и настроим нашу среду.

In [ ]:
import os
import json
import random
import time
import numpy as np
from typing import List, Annotated, TypedDict, Optional, Literal
from dotenv import load_dotenv

# Pydantic для моделирования состояния мира
from pydantic import BaseModel, Field

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - Simulator (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
)

In [ ]:
import os
import random
import numpy as np
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic для моделирования данных
from pydantic import BaseModel, Field

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown
from rich.table import Table

# --- Настройка API ключей и трассировки ---
load_dotenv(dotenv_path="../.env")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Simulator (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Переменные окружения загружены и трассировка настроена.")

Environment variables loaded and tracing is set up.


## Фаза 1: Построение среды Симулятора

Сначала нам нужно создать "мир", с которым будет взаимодействовать наш агент. Мы построим класс `MarketSimulator`, который управляет состоянием акции, портфелем и включает функцию `step` для продвижения времени. Это будет служить как нашим "реальным миром", так и песочницей для симуляций нашего агента.

In [ ]:
console = Console()

class Portfolio(BaseModel):
    cash: float = 10000.0
    shares: int = 0
    
    def value(self, current_price: float) -> float:
        return self.cash + self.shares * current_price

class MarketSimulator(BaseModel):
    """Простая симуляция фондового рынка для одного актива."""
    day: int = 0
    price: float = 100.0
    volatility: float = 0.1 # Стандартное отклонение для изменений цены
    drift: float = 0.01 # Общий тренд
    market_news: str = "Market is stable."
    portfolio: Portfolio = Field(default_factory=Portfolio)

    def step(self, action: str, amount: float = 0.0):
        """Продвигает симуляцию на один день, сначала выполняя сделку."""
        # 1. Выполнение сделки
        if action == "buy": # amount - это количество акций
            shares_to_buy = int(amount)
            cost = shares_to_buy * self.price
            if self.portfolio.cash >= cost:
                self.portfolio.shares += shares_to_buy
                self.portfolio.cash -= cost
        elif action == "sell": # amount - это количество акций
            shares_to_sell = int(amount)
            if self.portfolio.shares >= shares_to_sell:
                self.portfolio.shares -= shares_to_sell
                self.portfolio.cash += shares_to_sell * self.price
        
        # 2. Обновление рыночной цены (Геометрическое Броуновское движение)
        daily_return = np.random.normal(self.drift, self.volatility)
        self.price *= (1 + daily_return)
        
        # 3. Продвижение времени
        self.day += 1
        
        # 4. Потенциальное обновление новостей
        if random.random() < 0.1: # 10% шанс появления новых новостей
            self.market_news = random.choice(["Positive earnings report expected.", "New competitor enters the market.", "Macroeconomic outlook is strong.", "Regulatory concerns are growing."])
            # Новости влияют на тренд (drift)
            if "Positive" in self.market_news or "strong" in self.market_news:
                self.drift = 0.05
            else:
                self.drift = -0.05
        else:
             self.drift = 0.01 # Возврат к нормальному тренду

    def get_state_string(self) -> str:
        return f"Day {self.day}: Price=${self.price:.2f}, News: {self.market_news}\nPortfolio: ${self.portfolio.value(self.price):.2f} ({self.portfolio.shares} shares, ${self.portfolio.cash:.2f} cash)"

print("Среда симулятора рынка успешно определена.")

Market simulator environment defined successfully.


## Фаза 2: Создание Агента-Симулятора

Теперь мы используем LangGraph для оркестрации рабочего процесса `Наблюдение -> Предложение -> Симуляция -> Уточнение -> Выполнение`. Мы определим модели Pydantic для выходных данных LLM, чтобы обеспечить структурированную коммуникацию между шагами.

In [ ]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.4)

# Модели Pydantic для структурированных выходных данных LLM
class ProposedAction(BaseModel):
    """Высокоуровневая стратегия, предложенная аналитиком.""" 
    strategy: str = Field(description="Высокоуровневая торговая стратегия, например, 'buy aggressively', 'sell cautiously', 'hold'.")
    reasoning: str = Field(description="Краткое обоснование предложенной стратегии.")

class FinalDecision(BaseModel):
    """Окончательное конкретное действие для выполнения."""
    action: str = Field(description="Финальное действие: 'buy', 'sell' или 'hold'.")
    amount: float = Field(description="Количество акций для покупки или продажи. Должно быть 0 при удержании.")
    reasoning: str = Field(description="Окончательное обоснование со ссылкой на результаты симуляции.")

# Состояние LangGraph
class AgentState(TypedDict):
    real_market: MarketSimulator
    proposed_action: Optional[ProposedAction]
    simulation_results: Optional[List[Dict]]
    final_decision: Optional[FinalDecision]

# Узлы графа
def propose_action_node(state: AgentState) -> Dict[str, Any]:
    """Наблюдает за рынком и предлагает высокоуровневую стратегию."""
    console.print("--- 🧐 Аналитик предлагает действие (Analyst) ---")
    prompt = ChatPromptTemplate.from_template(
        "You are a sharp financial analyst. Based on the current market state, propose a trading strategy.\n\nMarket State:\n{market_state}"
    )
    proposer_llm = llm.with_structured_output(ProposedAction)
    chain = prompt | proposer_llm
    proposal = chain.invoke({"market_state": state['real_market'].get_state_string()})
    console.print(f"[yellow]Предложение:[/yellow] {proposal.strategy}. [italic]Причина: {proposal.reasoning}[/italic]")
    return {"proposed_action": proposal}

def run_simulation_node(state: AgentState) -> Dict[str, Any]:
    """Запускает предложенную стратегию в изолированной симуляции."""
    console.print("--- 🤖 Запуск Симуляций (Simulations) ---")
    strategy = state['proposed_action'].strategy
    num_simulations = 5
    simulation_horizon = 10 # дней
    results = []

    for i in range(num_simulations):
        # ВАЖНО: Создаем глубокую копию, чтобы не повлиять на состояние реального рынка
        simulated_market = state['real_market'].model_copy(deep=True)
        initial_value = simulated_market.portfolio.value(simulated_market.price)

        # Перевод стратегии в конкретное действие для симуляции
        if "buy" in strategy:
            action = "buy"
            # Aggressively (агрессивно) = 25% наличных, Cautiously (осторожно) = 10%
            amount = (simulated_market.portfolio.cash * (0.25 if "aggressively" in strategy else 0.1)) / simulated_market.price
        elif "sell" in strategy:
            action = "sell"
            # Aggressively (агрессивно) = 25% акций, Cautiously (осторожно) = 10%
            amount = simulated_market.portfolio.shares * (0.25 if "aggressively" in strategy else 0.1)
        else:
            action = "hold"
            amount = 0
        
        # Запуск симуляции вперед
        simulated_market.step(action, amount)
        for _ in range(simulation_horizon - 1):
            simulated_market.step("hold") # Просто держать после начального действия
        
        final_value = simulated_market.portfolio.value(simulated_market.price)
        results.append({"sim_num": i+1, "initial_value": initial_value, "final_value": final_value, "return_pct": (final_value - initial_value) / initial_value * 100})
    
    console.print("[cyan]Симуляция завершена. Результаты будут переданы риск-менеджеру.[/cyan]")
    return {"simulation_results": results}

def refine_and_decide_node(state: AgentState) -> Dict[str, Any]:
    """Анализирует результаты симуляции и принимает окончательное, уточненное решение."""
    console.print("--- 🧠 Риск-менеджер уточняет решение (Risk Manager) ---")
    results_summary = "\n".join([f"Sim {r['sim_num']}: Initial=${r['initial_value']:.2f}, Final=${r['final_value']:.2f}, Return={r['return_pct']:.2f}%" for r in state['simulation_results']])
    
    prompt = ChatPromptTemplate.from_template(
        "You are a cautious risk manager. Your analyst proposed a strategy. You have run simulations to test it. Based on the potential outcomes, make a final, concrete decision. If results are highly variable or negative, reduce risk (e.g., buy/sell fewer shares, or hold).\n\nInitial Proposal: {proposal}\n\nSimulation Results:\n{results}\n\nReal Market State:\n{market_state}"
    )
    decider_llm = llm.with_structured_output(FinalDecision)
    chain = prompt | decider_llm
    final_decision = chain.invoke({
        "proposal": state['proposed_action'].strategy,
        "results": results_summary,
        "market_state": state['real_market'].get_state_string()
    })
    console.print(f"[green]Финальное решение:[/green] {final_decision.action} {final_decision.amount:.0f} shares. [italic]Причина: {final_decision.reasoning}[/italic]")
    return {"final_decision": final_decision}

def execute_in_real_world_node(state: AgentState) -> Dict[str, Any]:
    """Выполняет финальное решение в среде реального рынка."""
    console.print("--- 🚀 Выполнение в Реальном Мире (Real World) ---")
    decision = state['final_decision']
    real_market = state['real_market']
    real_market.step(decision.action, decision.amount)
    console.print(f"[bold]Выполнение завершено. Новое состояние рынка:[/bold]\n{real_market.get_state_string()}")
    return {"real_market": real_market}

# Построение графа
workflow = StateGraph(AgentState)
workflow.add_node("propose", propose_action_node)
workflow.add_node("simulate", run_simulation_node)
workflow.add_node("refine", refine_and_decide_node)
workflow.add_node("execute", execute_in_real_world_node)

workflow.set_entry_point("propose")
workflow.add_edge("propose", "simulate")
workflow.add_edge("simulate", "refine")
workflow.add_edge("refine", "execute")
workflow.add_edge("execute", END)

simulator_agent = workflow.compile()
print("Граф агента с Симулятором в цикле успешно скомпилирован.")

Simulator-in-the-loop agent graph compiled successfully.


## Фаза 3: Демонстрация

Давайте запустим нашего агента на несколько дней на рынке. Мы начнем с хороших новостей и посмотрим, как он отреагирует, затем введем плохие новости.

In [ ]:
real_market = MarketSimulator()
console.print("--- Начальное состояние рынка ---")
console.print(real_market.get_state_string())

# --- Запуск День 1 ---
console.print("\n--- День 1: Хорошие новости! ---")
real_market.market_news = "Positive earnings report expected."
real_market.drift = 0.05
initial_state = {"real_market": real_market}
final_state = simulator_agent.invoke(initial_state)
real_market = final_state['real_market']

# --- Запуск День 2 ---
console.print("\n--- День 2: Плохие новости! ---")
real_market.market_news = "New competitor enters the market."
real_market.drift = -0.05
initial_state = {"real_market": real_market}
final_state = simulator_agent.invoke(initial_state)
real_market = final_state['real_market']

--- Initial Market State ---


Day 0: Price=$100.00, News: Market is stable.
Portfolio: $10000.00 (0 shares, $10000.00 cash)



---  Day 1: Good News Hits! ---


--- 🧐 Analyst Proposing Action ---
Proposal: buy aggressively. Reason: The positive earnings report is a strong bullish signal, and the market is already stable. This is a good opportunity to enter a position before the price potentially rises further.
--- 🤖 Running Simulations ---
Simulation complete. Results will be passed to the risk manager.
--- 🧠 Risk Manager Refining Decision ---
Final Decision: buy 20 shares. Reason: The simulations confirm a strong upward trend, with all scenarios resulting in a positive return. The analyst's proposal to buy aggressively is validated. I will execute a significant but not excessive purchase of 20 shares to capitalize on the expected price increase while maintaining a cash reserve.
--- 🚀 Executing in Real World ---
Execution complete. New market state:
Day 1: Price=$99.16, News: Market is stable.
Portfolio: $7983.18 (20 shares, $8000.00 cash)



--- Day 2: Bad News Hits! ---


--- 🧐 Analyst Proposing Action ---
Proposal: sell cautiously. Reason: The entry of a new competitor introduces significant uncertainty and potential downside risk. While the price hasn't dropped dramatically yet, it's prudent to reduce exposure.
--- 🤖 Running Simulations ---
Simulation complete. Results will be passed to the risk manager.
--- 🧠 Risk Manager Refining Decision ---
Final Decision: sell 5 shares. Reason: The simulations show a high degree of variance and a negative average return, confirming the analyst's concerns. The initial proposal to sell cautiously is sound. I will de-risk the portfolio by selling 5 shares (25% of the position) to lock in some cash and reduce exposure to the potential downside from the new competitor.
--- 🚀 Executing in Real World ---
Execution complete. New market state:
Day 2: Price=$93.81, News: Market is stable.
Portfolio: $9802.90 (15 shares, $8395.73 cash)


### Анализ результатов

Поведение агента демонстрирует ценность цикла симуляции:

- **В, день 1 (Хорошие новости):**
    - *Аналитик (Analyst)* предложил агрессивную покупку, увидев возможность.
    - *Симулятор (Simulator)* подтвердил высокую вероятность положительного исхода.
    - *Риск-менеджер (Risk Manager)* перевел агрессивную стратегию в конкретную, существенную покупку (20 акций), но не рискнул всем балансом наличных.

- **В день 2 (Плохие новости):**
    - *Аналитик* правильно определил новый риск и предложил осторожную продажу.
    - *Симулятор*, вероятно, показал смешанные результаты: некоторые сценарии показывали резкое падение, другие — восстановление, подтверждая неопределенность.
    - *Риск-менеджер*, видя разброс и отрицательную среднюю доходность в симуляциях, принял разумное решение сократить позицию (продав 5 акций), вместо того чтобы панически продавать всё. Это гораздо более нюансированное действие, чем мог бы предпринять простой агент на основе правил.

Наивный агент без цикла симуляции мог бы купить слишком много в первый день и затем продать всё во второй, неся более высокие транзакционные издержки и потенциально упуская восстановление. Наш агент-симулятор действовал больше как трейдер в реальном мире, делая вероятностную ставку, а затем хеджируя эту ставку, когда новая информация меняла профиль риска.

## Заключение

В этом блокноте мы построили мощную архитектуру агента, которая использует внутренний **симулятор** для тестирования и уточнения своих действий перед их совершением. Создав цикл `Предложение -> Симуляция -> Уточнение -> Выполнение`, мы позволили нашему агенту выполнять сложный анализ рисков и принимать более тонкие, безопасные решения в динамичной среде.

Этот паттерн является краеугольным камнем создания агентов, которые могут безопасно и эффективно работать в реальном мире. Способность выполнять анализ "что-если" на внутренней "ментальной модели" позволяет агенту предвидеть последствия, избегать дорогостоящих ошибок и разрабатывать более надежные стратегии. Хотя точность симулятора является критической зависимостью ("разрыв между симуляцией и реальностью"), эта архитектура предоставляет четкий и расширяемый фреймворк для создания ответственного, действующего ИИ.